# Build Treasury Quotes CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()


In [2]:
db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [3]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [4]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')

In [5]:
DATE = '2024-04-30'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204083.0,20241115.107500,2024-04-30,101.289062,101.320312,101.304688,M,I,3.440934,-0.000027,0.000135,192.425922,3478.0,9604.0,0.0,<NA>,<NA>
1,204084.0,20250215.107620,2024-04-30,102.070312,102.101562,102.085938,M,I,1.571085,-0.000175,0.000132,284.327462,3333.0,9509.0,0.0,<NA>,<NA>
2,204085.0,20250815.106870,2024-04-30,102.484375,102.515625,102.5,M,I,1.416552,-0.000344,0.000131,454.331736,3356.0,11187.0,0.0,<NA>,<NA>
3,204086.0,20260215.106000,2024-04-30,101.851562,101.882812,101.867188,M,I,1.236264,0.001754,0.000132,625.000942,6194.0,12838.0,0.0,<NA>,<NA>
4,204087.0,20260815.106750,2024-04-30,103.726562,103.757812,103.742188,M,I,1.390797,-0.000715,0.000135,780.741606,3670.0,8810.0,0.0,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,208213.0,20260430.204870,2024-04-30,99.679688,99.714844,99.697266,M,I,0.0,-0.00092,0.000136,704.384067,<NA>,69319.0,0.0,<NA>,<NA>
439,208214.0,20270415.204500,2024-04-30,98.953125,98.984375,98.96875,M,I,0.184426,-0.001764,0.000132,1021.167754,<NA>,58951.0,0.0,<NA>,<NA>
440,208215.0,20290415.C02120,2024-04-30,99.199219,99.269531,99.234375,M,I,0.0,<NA>,<NA>,<NA>,<NA>,23171.0,0.0,1.00309,C
441,208216.0,20290430.204620,2024-04-30,99.574219,99.621094,99.597656,M,I,0.0,-0.003089,0.000128,1650.725702,<NA>,70333.0,0.0,<NA>,<NA>


In [6]:
# drop any NaNs, cast to int, pull out unique values
ids = (data_quotes['kytreasno']
         .dropna()
         .astype(int)
         .unique()
         .tolist())

id_list = ','.join(str(i) for i in ids)
SQL_QUERY = (
    "select * "
    "from crsp_m_treasuries.tfz_iss "
    f"where kytreasno in ({id_list})"
)
data_iss = db.raw_sql(SQL_QUERY)

In [7]:
# KEYS = tuple(data_quotes['kytreasno'])
# SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
# data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,20241115.107500,912810ES,1994-05-15,2024-11-15,1.0,7.5,2.0,3.75,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
1,204084.0,20250215.107620,20250215.107620,912810ET,1995-02-15,2025-02-15,1.0,7.625,2.0,3.8125,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
2,204085.0,20250815.106870,20250815.106870,912810EV,1995-08-15,2025-08-15,0.0,6.875,2.0,3.4375,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
3,204086.0,20260215.106000,20260215.106000,912810EW,1996-02-15,2026-02-15,0.0,6.0,2.0,3.0,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
4,204087.0,20260815.106750,20260815.106750,912810EX,1996-08-15,2026-08-15,0.0,6.75,2.0,3.375,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,208213.0,20260430.204870,20260430.204870,91282CKK,2024-04-30,2026-04-30,0.0,4.875,2.0,2.4375,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
439,208214.0,20270415.204500,20270415.204500,91282CKJ,2024-04-15,2027-04-15,0.0,4.5,2.0,2.25,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
440,208215.0,20290415.C02120,20290415.C02120,91282CKL,2024-04-15,2029-04-15,0.0,2.125,2.0,1.0625,...,<NA>,0.0,<NA>,12.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
441,208216.0,20290430.204620,20290430.204620,91282CKP,2024-04-30,2029-04-30,0.0,4.625,2.0,2.3125,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>


In [8]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')
rawdata

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,2024-04-30,101.289062,101.320312,101.304688,M,I,3.440934,-0.000027,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
1,204084.0,20250215.107620,2024-04-30,102.070312,102.101562,102.085938,M,I,1.571085,-0.000175,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
2,204085.0,20250815.106870,2024-04-30,102.484375,102.515625,102.5,M,I,1.416552,-0.000344,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
3,204086.0,20260215.106000,2024-04-30,101.851562,101.882812,101.867188,M,I,1.236264,0.001754,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
4,204087.0,20260815.106750,2024-04-30,103.726562,103.757812,103.742188,M,I,1.390797,-0.000715,...,<NA>,0.0,<NA>,1.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,208213.0,20260430.204870,2024-04-30,99.679688,99.714844,99.697266,M,I,0.0,-0.00092,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
439,208214.0,20270415.204500,2024-04-30,98.953125,98.984375,98.96875,M,I,0.184426,-0.001764,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
440,208215.0,20290415.C02120,2024-04-30,99.199219,99.269531,99.234375,M,I,0.0,<NA>,...,<NA>,0.0,<NA>,12.0,0.0,1.0,1.0,<NA>,<NA>,<NA>
441,208216.0,20290430.204620,2024-04-30,99.574219,99.621094,99.597656,M,I,0.0,-0.003089,...,<NA>,0.0,<NA>,2.0,0.0,1.0,1.0,<NA>,<NA>,<NA>


***

# Process Data

In [9]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data, select_maturities

data = process_wrds_treasury_data(rawdata,keys_extra=['public size'])

In [10]:
idx = select_maturities(data)
data_select = data.loc[idx]

***

# Save to Excel

In [11]:
### raw data
outfile = f'../data/treasury_quotes_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    rawdata.columns = rawdata.columns.str.upper()
    rawdata.to_excel(writer, sheet_name= 'quotes', index=False)


### processed data
outfile = f'../data/treasury_quotes_clean_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=True)
    data_select.to_excel(writer, sheet_name = 'selected quotes', index=True)